In [3]:
import camelot
import json
import re
import datetime
import pandas as pd

from bs4 import BeautifulSoup
import requests
import datetime

In [4]:
ru_url = 'https://ru.unb.br'
menu_url = ru_url + '/index.php/cardapio-refeitorio'

html_menu_response = requests.get(menu_url)

In [5]:
def onDay(date, day):
    """
    Returns the date of the next given weekday after
    the given date. For example, the date of next Monday.

    NB: if it IS the day we're looking for, this returns 0.
    consider then doing onDay(foo, day + 1).
    """
    days = (day - date.weekday() + 7) % 7
    return date + datetime.timedelta(days=days)


menu_soup = BeautifulSoup(html_menu_response.text, 'html')

campuses_names = []
campuses = menu_soup('strong')
for campus in campuses:
   if isinstance(campus.contents[0], str):
    campuses_names.append(campus.string)
print(campuses_names)

available_menu_links = menu_soup.find('main').find_all('a')
tables = {}
idx = 0
d = datetime.datetime.now()
next_monday = onDay(d, 0).strftime('%d/%m').replace('/', '-')
for item in available_menu_links:
    if item.contents and item.string != 'I' and next_monday in item.string: # m.contents[0] != 'I', isso é para desconsiderar um link nada a ver no site
      menu_link = ru_url + item.attrs['href']
      print('Processando cardapio: ' + menu_link)
      tables[campuses_names[idx]] = camelot.read_pdf(menu_link, pages='1,2,3', flavor='lattice', line_scale=40, strip_text='\n', copy_text=['h'], )
      idx += 1
tables

['Darcy Ribeiro', 'Ceilândia', 'Gama', 'Planaltina', 'Fazenda']
Processando cardapio: https://ru.unb.br/images/Artigos/ISMCARDAPIOSDRU2023/11NOVEMBRO2023/Darcy_Ribeiro-_Semana_02_-_20-11_A_26-11.pdf
Processando cardapio: https://ru.unb.br/images/Artigos/ISMCARDAPIOSDRU2023/11NOVEMBRO2023/Ceilndia-_Semana_02_-_20-11_A_26-11_-_Copia_-_Copia.pdf
Processando cardapio: https://ru.unb.br/images/Artigos/ISMCARDAPIOSDRU2023/11NOVEMBRO2023/Gama-_Semana_02_-_20-11_A_26-11.pdf
Processando cardapio: https://ru.unb.br/images/Artigos/ISMCARDAPIOSDRU2023/11NOVEMBRO2023/Planaltina-_Semana_02_-_20-11_A_26-11_-_Copia.pdf
Processando cardapio: https://ru.unb.br/images/Artigos/ISMCARDAPIOSDRU2023/11NOVEMBRO2023/Fazenda-_Semana_02_-_20-11_A_26-11.pdf


{'Darcy Ribeiro': <TableList n=3>,
 'Ceilândia': <TableList n=3>,
 'Gama': <TableList n=3>,
 'Planaltina': <TableList n=3>,
 'Fazenda': <TableList n=3>}

In [6]:
def table_to_json(table):
  for i in range(3):
    table[i].df = table[i].df.drop(0, axis=1) #removes first column
    table[i].df.index = table[i].df[1]
    table[i].df = table[i].df.drop(1, axis=1) #removes first column

  dates = re.findall("\d\d.\d\d.\d\d\d\d", table[0].df.to_string())
  formated_dates = map(lambda date: datetime.datetime.strptime(date, "%d/%m/%Y").strftime("%Y-%m-%d"), dates)
  dates = list(formated_dates)

  str_list = list(filter(None, table[0].df.T.columns[1:]))
  x = table[0].df.T.values[i][1:].tolist()
  try:
    x.remove('Café OU chá')
  except:
    try:
      x.remove('Café ou chá')
    except:
      pass

  desjejum = []
  almoco = []
  jantar = []
  for j in range(3):
    for i in range(len(table[j].df.columns.values)):
      if j == 0:
        desjejum_list = table[0].df.T.values[i][1:].tolist()
        try:
          desjejum_list.remove('Café OU chá')
        except:
          try:
            desjejum_list.remove('Café ou chá')
          except:
            pass
        desjejum.append(json.dumps(dict(zip(map(lambda x: x.capitalize(),list(filter(None, table[0].df.T.columns[1:]))), desjejum_list)), ensure_ascii=False))
      elif j == 1:
        almoco.append(json.dumps(dict(zip(map(lambda x: x.capitalize(),table[1].df.T.columns[1:]), table[1].df.T.values[i][1:])), ensure_ascii=False))
      elif j == 2:
        jantar.append(json.dumps(dict(zip(map(lambda x: x.capitalize(), table[2].df.T.columns[1:]), table[2].df.T.values[i][1:])), ensure_ascii=False))
  
  print(dict(date = dates, desjejum = desjejum, almoco = almoco, jantar = jantar))
  return  pd.DataFrame({'date': dates, 'desjejum': desjejum, 'almoco': almoco, "jantar": jantar})

In [7]:
for campus in tables:
  res = table_to_json(tables[campus])
  res.to_csv(campus + ".csv", index=False)

{'date': ['2023-11-20', '2023-11-21', '2023-11-22', '2023-11-23', '2023-11-24', '2023-11-25', '2023-11-26'], 'desjejum': ['{"Bebidas": "Leite integral ou “leite” de soja", "Panificação": "Pão francês  ou Pão careca  ou Pão integral", "Opção extra": "", "Gordura": "Manteiga ou creme vegetal", "Complemento padrão": "Salsicha ao molho", "Complemento ovolactovegetariano  queijo muçarela": "", "Complemento vegetariano estrito": "Amentondela", "Fruta": "Melancia"}', '{"Bebidas": "Leite integral ou “leite” de soja", "Panificação": "Pão francês  ou Pão careca  ou", "Opção extra": "Cuscuz", "Gordura": "Manteiga ou creme vegetal", "Complemento padrão": "Carne desfiada", "Complemento ovolactovegetariano  queijo muçarela": "Ovos mexidos", "Complemento vegetariano estrito": "Geleia de frutas", "Fruta": "Mamão"}', '{"Bebidas": "Leite integral ou “leite” de soja", "Panificação": "Pão francês  ou Pão careca  ou", "Opção extra": "", "Gordura": "Manteiga ou creme vegetal", "Complemento padrão": "Queijo 

OSError: Cannot save file into a non-existent directory: 'output'